# Dependecies and libraries loading
here we load all of the dependecies and libraries we will need for the project

In [1]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
from sklearn.model_selection import train_test_split #to split out training and testing data 
import torch
#for command line arguments
import argparse
#for reading files
import os
import cv2, os
import numpy as np
import matplotlib.image as mpimg
#for modeling
import torchvision, torch
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from skimage import io, transform
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import time
np.random.seed(0)
print('done')

done


# Nvidia's Model:
 NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.<br>
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU<br>
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU<br>
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU<br>
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU<br>
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU<br>
    Drop out (0.5)<br>
    Fully connected: neurons: 100, activation: ELU<br>
    Fully connected: neurons: 50, activation: ELU<br>
    Fully connected: neurons: 10, activation: ELU<br>
    Fully connected: neurons: 1 (output)<br>

    the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient     problem. 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=10
batch_size=32
learning_rate=0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, 5, stride=2)
        self.conv2 = nn.Conv2d(24, 36, 5, stride=2)
        self.conv3 = nn.Conv2d(36, 48, 5, stride=2)
        self.conv4 = nn.Conv2d(48, 64, 3)
        self.conv5 = nn.Conv2d(64, 64, 3)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(1088, 100) # This might need to be adjusted depending on the input image size
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 1)
    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = F.elu(self.conv5(x))
        x = self.dropout(x)
        x = x.view(x.size(0), -1) # This flattens the output of the conv layers
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = self.fc4(x) # No activation function is used for the output layer
        return x

model = Net().to(device)
criterion=nn.MSELoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)
print("done")

done


# The DataLoader
This DataLoader was made using pytorch DataLoader class it has an intialization __init__() where we would define the inputs of the dataloader a __len()__ function to retain the length of the data input a __getitem()__ function to transform and return the data to be ready to use  

In [3]:
class cam (torch.utils.data.Dataset):
    """cam dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.cam_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.img_name= self.cam_frame.iloc[:,0]
        self.transform = transform
        self.y = self.cam_frame.iloc[:, 3]

    def __len__(self):
        return len(self.cam_frame)

    def __getitem__(self, index):
        img=Image.open(os.path.join(self.root_dir,
                                self.img_name[index]))

        if self.transform:
            sample = self.transform(img)
        label = self.y[index]
        return sample,label

# DataLoading
Here we load the data using the Data loader class

In [4]:
transform_car = transforms.Compose(
        [transforms.Resize([66, 200]),
         transforms.RandomCrop([62,194]),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor()])
train_dataset = cam(csv_file="brbr\driving_log.csv",root_dir = "brbr\IMG",transform=transform_car )
train_loader=DataLoader(dataset=train_dataset,batch_size=32,drop_last=True,shuffle=True,num_workers=0)

In [5]:
valid_dataset =cam(csv_file="valid\driving_log.csv",root_dir = "valid\IMG",transform=transform_car )
valid_loader=DataLoader(dataset=valid_dataset,batch_size=100,shuffle=False,num_workers=0)

# Model training and validation 
here we train the model using a __train_model()__ function made to train and validate the model and save the model with best validation result

In [6]:
def train_model(model, num_epochs, train_loader,
                valid_loader, optimizer, device):
    min_valid_loss=7.8
    start_time = time.time()
    minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []
    for epoch in range(num_epochs):

        model.train()
        for batch_idx, data in enumerate(train_loader):

            features,targets = data
            features = features.to(device)
            targets = targets.to(device)

            # ## FORWARD AND BACK PROP
            logits = model(features)
            loss = criterion(logits, targets.to(torch.float32))
            optimizer.zero_grad()

            loss.backward()

            # ## UPDATE MODEL PARAMETERS
            optimizer.step()

            # ## LOGGING
            minibatch_loss_list.append(loss.item())
            if not batch_idx % 50:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}')

        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for data, labels in valid_loader:
            data, labels = data.to(device), labels.to(device)
        
            target = model(data)
            loss = criterion(target,labels)
            valid_loss = loss.item() * data.size(0)

            print(f'Epoch {epoch+1:03d}/{num_epochs:03d} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(model.state_dict(), 'saved_model.pth')

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    return minibatch_loss_list

In [7]:
minibatch_loss_list=train_model(model=model,num_epochs=50,train_loader=train_loader,
                                                             valid_loader=valid_loader,test_loader=test_loader,optimizer=optimizer,device=device)

Epoch: 001/050 | Batch 0000/0135 | Loss: 0.0246


C:\Users\ahmed\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 001/050 | Batch 0050/0135 | Loss: 0.0076
Epoch: 001/050 | Batch 0100/0135 | Loss: 0.0046


C:\Users\ahmed\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 001/050 		 Validation Loss: 0.17423808192013193
Validation Loss Decreased(7.800000--->3.136285) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.07829411649039464
Validation Loss Decreased(3.136285--->1.409294) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.005530158263077385
Validation Loss Decreased(1.409294--->0.099543) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.008386480879168358
Epoch 001/050 		 Validation Loss: 0.010892375991888817
Epoch 001/050 		 Validation Loss: 0.007703802295979608
Epoch 001/050 		 Validation Loss: 0.07080148367387055
Epoch 001/050 		 Validation Loss: 0.19475314888959377
Epoch 001/050 		 Validation Loss: 0.05345049821584299
Epoch 001/050 		 Validation Loss: 0.008968326013580758
Epoch 001/050 		 Validation Loss: 0.012730339542124804
Epoch 001/050 		 Validation Loss: 0.006779453889349838
Epoch 001/050 		 Validation Loss: 0.012725177781531385
Epoch 001/050 		 Validation Loss: 0.011791832270307287
Epoch 001/050 		 Validation Loss: 

C:\Users\ahmed\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([74])) that is different to the input size (torch.Size([74, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 002/050 | Batch 0000/0135 | Loss: 0.0047
Epoch: 002/050 | Batch 0050/0135 | Loss: 0.0043
Epoch: 002/050 | Batch 0100/0135 | Loss: 0.0074
Epoch 002/050 		 Validation Loss: 0.1663339469480799
Epoch 002/050 		 Validation Loss: 0.07992562574066896
Epoch 002/050 		 Validation Loss: 0.008645024895155971
Epoch 002/050 		 Validation Loss: 0.0076782857378785525
Epoch 002/050 		 Validation Loss: 0.01011115840165711
Epoch 002/050 		 Validation Loss: 0.010777803154641444
Epoch 002/050 		 Validation Loss: 0.06778015757613677
Epoch 002/050 		 Validation Loss: 0.20499909982219586
Epoch 002/050 		 Validation Loss: 0.047814543356791486
Epoch 002/050 		 Validation Loss: 0.010841830568858615
Epoch 002/050 		 Validation Loss: 0.01571754505206495
Epoch 002/050 		 Validation Loss: 0.00594261731391284
Epoch 002/050 		 Validation Loss: 0.011840449420083166
Epoch 002/050 		 Validation Loss: 0.014916061584277766
Epoch 002/050 		 Validation Loss: 0.08103511032361291
Epoch 002/050 		 Validation Loss: 0.122